In [1]:
import pandas as pd

## Erstelle aus `bank-churn.csv` einen DataFrame

In [2]:
df = pd.read_csv('data/bank-churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data Cleaning

In [3]:
df = df.drop(columns=["CustomerId", "Surname", "RowNumber"])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df = pd.get_dummies(data=df, columns=['Gender', 'Geography'], drop_first=True)
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


## Training
* 20% Testdaten
* `random_state=42` sofern unterstützt
* target feature: `Exited`
* Ziel: score von 0.85

In [5]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='Exited')
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
9254,686,32,6,0.00,2,1,1,179093.26,1,0,0
1561,632,42,4,119624.60,2,1,1,195978.86,1,1,0
1670,559,24,3,114739.92,1,1,0,85891.02,1,0,1
6087,561,27,9,135637.00,1,1,0,153080.40,0,0,0
6669,517,56,9,142147.32,1,0,0,39488.04,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5734,768,54,8,69712.74,1,1,1,69381.05,1,0,0
5191,682,58,1,0.00,1,1,1,706.50,0,0,0
5390,735,38,1,0.00,3,0,0,92220.12,0,0,0
860,667,43,8,190227.46,1,1,0,97508.04,1,0,0


In [6]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

ann = Sequential([
    layers.Dense(units=11, activation='relu', dtype='float64'),
    layers.Dense(units=11, activation='relu', dtype='float64'),
    layers.Dense(units=1, activation='sigmoid', dtype='float64'),   # LogisticRegression
])

ann.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
history = ann.fit(X_train,
                  y_train,
                  batch_size=16,
                  validation_data=(X_test, y_test),
                  epochs=100,
                  verbose=1)

AlreadyExistsError: Another metric with the same name already exists.

In [25]:
import tensorflow as tf

ann = tf.keras.models.load_model('ann')

In [26]:
from sklearn import metrics

metrics.accuracy_score(ann.predict(X_test) > 0.5, y_test)

63/63 [==============================] - 0s 2ms/step


0.804

## Interpretiere den rechten oberen Eintrag der Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix
predictions = ann.predict(X_test) > 0.5
confusion_matrix(y_test, predictions)

63/63 [==============================] - 0s 2ms/step


array([[1607,    0],
       [ 392,    1]], dtype=int64)

Wir sagen 78 Customer leaven obwohl sie eigentlich bleiben

## Predicte, mit welcher Wahrscheinlichkeit der Kunde aussteigen wird
Er sollte mit etwa 90%iger Wahrscheinlichkeit **bleiben**

In [10]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Male,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [17]:
customer = pd.DataFrame(data={
    'CreditScore': 500,
    'Age': 20,
    'Tenure': 2,
    'Balance': 0.0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60_000.0,
    'Geography': 'Germany',
    'Gender': 'Male'},
    index=[0])

In [18]:
customer = customer.drop(columns=["Gender", "Geography"])
customer["Gender_Male"] = 1
customer["Geography_Germany"] = 1
customer["Geography_Spain"] = 0
customer

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_Germany,Geography_Spain
0,500,20,2,0.0,1,1,1,60000.0,1,1,0


In [28]:
1 - ann.predict(customer)

1/1 [==============================] - 0s 26ms/step


array([[0.99999918]])

In [29]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(X_train, y_train)
forest.score(X_test, y_test)

0.865

In [30]:
1 - forest.predict(customer)

array([1], dtype=int64)